In [5]:
import pandas as pd
import glob
import shutil
import sys
import numpy as np
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import math
from sklearn.impute import SimpleImputer

In [2]:
df = pd.read_pickle('df.pkl')

In [ ]:
df.to_pickle('df.pkl')

In [7]:
imputer = SimpleImputer(copy=False, fill_value=None, strategy='most_frequent', verbose=0)

In [8]:
# Remove the rows, where the value count is less than 1000
print(df.Census_InternalBatteryType.unique())
df['Census_InternalBatteryType'] = imputer.fit_transform(df['Census_InternalBatteryType'])
print(df.Census_InternalBatteryType.unique())

[nan 'lion' 'li-i' '#' 'lip' 'liio' 'vbox' 'li p' 'real' 'unkn' 'pbac'
 'li' 'bq20' 'nimh' '\x04lio' 'lgi0' 'lhp0' 'ithi' 'batt' 'lipp' 'lipo'
 '4cel' 'ram' 'lit' 'a140' 'bad' 'asmb' 'virt' 'ca48' '4ion' 'd' 'a132'
 'ÿÿÿÿ' 'cl53' 'lio' 'li-l' '÷ÿóö' 'í\x03-i' '0x0b' 'lgs0' '3ion' 'ots0'
 'lai0' 'lilo' 'pa50' 'h4°s' '5nm1' 'li-p' 'lhpo' '0ts0' 'pad0' 'sail'
 'p-sn' 'icp3' 'a130' '2337' '\x1f˙˙˙' 'lgl0' 'l\x15' '@i\uf8f5\uf8f5'
 'li\x90o' '4lio' 'lp' 'li?' '\x04ion' 'pbso' 'a138' 'li-h' '6ion' '3500'
 'h00j' 'li\x10' 'sams' '\x03ip' '8' '#TAB#' 'l\x06&#TAB#' 'liÿÿ' 'lÿÿÿ']


ValueError: Expected 2D array, got 1D array instead:
array=[nan nan nan ... nan nan nan].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
# Percentage of nulls
print(type(df.isnull().sum()/df.shape[0]))
plt.figure()
# (df.isnull().sum()/df.shape[0]).plot.hist()
print((df.isnull().sum()/df.shape[0]).sort_values(ascending=False))

In [ ]:
# Remove the rows, where the value count is less than 1000
print(df.Census_InternalBatteryType.value_counts())
# vc = df.Census_InternalBatteryType.value_counts()
# u  = [i not in set(vc[vc<1000].index) for i in df['Census_InternalBatteryType']]
# df = df[u]
print(df.Census_InternalBatteryType.value_counts())

In [ ]:
df.drop(df[df.SmartScreen == '&#x01;'].index, inplace=True)
df.drop(df[df.SmartScreen == '&#x02;'].index, inplace=True)
df.drop(df[df.SmartScreen == '&#x03;'].index, inplace=True)
df.drop(df[df.SmartScreen == '0'].index, inplace=True)
df.drop(df[df.SmartScreen == '00000000'].index, inplace=True)
df.drop(df[df.SmartScreen == 'Enabled'].index, inplace=True)
print(df['SmartScreen'].unique())
df.SmartScreen.value_counts()

In [ ]:
print(df['SmartScreen'].unique())
df.SmartScreen.value_counts()

In [ ]:
# Percentage of nulls
print(type(df.isnull().sum()/df.shape[0]))
plt.figure()
# (df.isnull().sum()/df.shape[0]).plot.hist()
print((df.isnull().sum()/df.shape[0]).sort_values(ascending=False))
(df.isnull().sum()/df.shape[0]).sort_values().plot()
plt.show()

In [ ]:
for f in drop_list:
    print('Dropping: ' + col)
    df = df.drop(f, 1)

In [ ]:
# Remove columns with only 1 unique value
# Should do in the end also.
# TODO
# df.dropna(thresh=2)
for col in df:
    if(len(df.loc[:,col].unique()) == 1):
        print('Dropping: ' + col)
        drop_list.append(col)

In [ ]:
# Now analyze the next 5 features
# print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
# start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
# print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
# print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
# print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
# print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
# print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
# print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
# print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
# print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
# print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
# print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
# print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
df['Census_OSVersion_0'] = df['Census_OSVersion'].apply(break_and_get_part, position=0)
df['Census_OSVersion_1'] = df['Census_OSVersion'].apply(break_and_get_part, position=1)
df['Census_OSVersion_2'] = df['Census_OSVersion'].apply(break_and_get_part, position=2)
df['Census_OSVersion_3'] = df['Census_OSVersion'].apply(break_and_get_part, position=3)

drop_list.append('Census_OSVersion')

print(df['Census_OSVersion_0'].unique())
print(df['Census_OSVersion_1'].unique())
print(df['Census_OSVersion_2'].unique())
print(df['Census_OSVersion_3'].unique())

In [ ]:
# Now analyze the next 5 features
print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
df.loc[df.SmartScreen == 'off', 'SmartScreen'] = "Off"
df.loc[df.SmartScreen == 'on', 'SmartScreen'] = "On"
df.loc[df.SmartScreen == 'OFF', 'SmartScreen'] = "Off"
df.loc[df.SmartScreen == 'requireAdmin', 'SmartScreen'] = "RequireAdmin"
df.loc[df.SmartScreen == 'Promt', 'SmartScreen'] = "Prompt"
df.loc[df.SmartScreen == 'prompt', 'SmartScreen'] = "Prompt"
df.loc[df.SmartScreen == 'warn', 'SmartScreen'] = "Warn"
df.loc[df.SmartScreen == 'requireadmin', 'SmartScreen'] = "RequireAdmin"
print(df['SmartScreen'].unique())

In [ ]:
# Now analyze the next 5 features
print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# print(df['OsBuildLab'].dtype)
# print(df['OsBuildLab'].unique())
df['OsBuildLab_0'] = df['OsBuildLab'].apply(break_and_get_part_osbuildlab, position=0)
df['OsBuildLab_1'] = df['OsBuildLab'].apply(break_and_get_part_osbuildlab, position=1)
df['OsBuildLab_2'] = df['OsBuildLab'].apply(break_and_get_part_osbuildlab, position=2)
df['OsBuildLab_3'] = df['OsBuildLab'].apply(break_and_get_part_osbuildlab, position=3)
df['OsBuildLab_4'] = df['OsBuildLab'].apply(break_and_get_part_osbuildlab, position=4)
df['OsBuildLab_5'] = df['OsBuildLab'].apply(break_and_get_part_osbuildlab, position=5)
drop_list.append('OsBuildLab')
print(df['OsBuildLab_0'].unique())
print(df['OsBuildLab_1'].unique())
print(df['OsBuildLab_2'].unique())
print(df['OsBuildLab_3'].unique())
print(df['OsBuildLab_4'].unique())
print(df['OsBuildLab_5'].unique())

In [ ]:
import math
def break_and_get_part_osbuildlab(value, position):
    if position < 4:
        try:
            return value.split(".")[position]
        except:
            return math.nan
    elif position == 4:
        try:
            return value.split(".")[4].split('-')[0]
        except:
            return math.nan
    elif position == 5:
        try:
            return value.split(".")[4].split('-')[1]
        except:
            return math.nan

In [ ]:
df['OsVer_0'] = df['OsVer'].apply(break_and_get_part, position=0)
df['OsVer_1'] = df['OsVer'].apply(break_and_get_part, position=1)
df['OsVer_2'] = df['OsVer'].apply(break_and_get_part, position=2)
df['OsVer_3'] = df['OsVer'].apply(break_and_get_part, position=3)
drop_list.append('OsVer')
print(df['OsVer_0'].unique())
print(df['OsVer_1'].unique())
print(df['OsVer_2'].unique())
print(df['OsVer_3'].unique())

In [ ]:
# Now analyze the next 5 features
print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
print(df['EngineVersion_0'].unique())
print(df['EngineVersion_1'].unique())
print(df['EngineVersion_2'].unique())
print(df['EngineVersion_3'].unique())
print(df['AppVersion_0'].unique())
print(df['AppVersion_1'].unique())
print(df['AppVersion_2'].unique())
print(df['AppVersion_3'].unique())
print(df['AvSigVersion_0'].unique())
print(df['AvSigVersion_1'].unique())
print(df['AvSigVersion_2'].unique())
print(df['AvSigVersion_3'].unique())

In [ ]:
df['EngineVersion_0'] = df['EngineVersion'].apply(break_and_get_part, position=0)
df['EngineVersion_1'] = df['EngineVersion'].apply(break_and_get_part, position=1)
df['EngineVersion_2'] = df['EngineVersion'].apply(break_and_get_part, position=2)
df['EngineVersion_3'] = df['EngineVersion'].apply(break_and_get_part, position=3)

df['AppVersion_0'] = df['AppVersion'].apply(break_and_get_part, position=0)
df['AppVersion_1'] = df['AppVersion'].apply(break_and_get_part, position=1)
df['AppVersion_2'] = df['AppVersion'].apply(break_and_get_part, position=2)
df['AppVersion_3'] = df['AppVersion'].apply(break_and_get_part, position=3)

df['AvSigVersion_0'] = df['AvSigVersion'].apply(break_and_get_part, position=0)
df['AvSigVersion_1'] = df['AvSigVersion'].apply(break_and_get_part, position=1)
df['AvSigVersion_2'] = df['AvSigVersion'].apply(break_and_get_part, position=2)
df['AvSigVersion_3'] = df['AvSigVersion'].apply(break_and_get_part, position=3)

drop_list.append('EngineVersion')
drop_list.append('AppVersion')
drop_list.append('AvSigVersion')

In [ ]:
def break_and_get_part(value, position):
    return value.split(".")[position]

In [ ]:
# Now analyze one column at a time
start_cnt = 0
show_cnt = 5
print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
    print(y, df[y].dtype)
start_cnt += show_cnt

In [ ]:
# Remove columns with only 1 unique value
# Should do in the end also.
# TODO
# df.dropna(thresh=2)
for col in df:
    if(len(df.loc[:,col].unique()) == 1):
        print('Dropping: ' + col)
        drop_list.append(col)

In [ ]:
# Drop MachineIdentifier
drop_list.append('MachineIdentifier')

In [ ]:
drop_list = []

import operator 
# Find all the NaN
# See what percentage of each column is NaN
x_pct = {}
length = len(df)
for y in df.columns:
    if df[y].isnull().sum() > 0:
        x_cnt = df[y].isnull().sum()
        x_pct[y] = 100.0*x_cnt/length
        
x_pct_list = sorted(x_pct.items(), key=operator.itemgetter(1), reverse=True)
      
# Print the features, where more than 80% of the data is NaN    
for f, val in x_pct_list:
    if val > 80:
        print(f, val)
        drop_list.append(f)
        print('Dropping: ' + f)
        print('------')

In [ ]:
pd.set_option('max_colwidth',1000)

In [ ]:
import matplotlib.pyplot as plt
# Percentage of nulls
print(type(df.isnull().sum()/df.shape[0]))
plt.figure()
# (df.isnull().sum()/df.shape[0]).plot.hist()
print((df.isnull().sum()/df.shape[0]).sort_values())
(df.isnull().sum()/df.shape[0]).sort_values().plot()
plt.show()

In [ ]:
# Delete columns, where all the entries are 0.
df = df.loc[:, (df != 0).any(axis=0)]
print(df.shape)
df.info()

In [ ]:
file = 'data/train.csv'
df = pd.read_csv(file)

In [ ]:
import pandas as pd
import glob
import shutil
import sys
import numpy as np
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import math
from sklearn.preprocessing import Imputer